<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c1138f1b7d3e69a32ebd5ed2467166b994f033e1b56a44b3991f303efe31432b
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-16 11:00:43
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.42 C
-------------------
Today PnL: 55.50 K (0.39%)
Current PnL: -17.02 L (-11.45%)
CY Booked + Current PnL: -6.00 L (-4.03%)
-------------------
Total profit:  2.18 L
Total loss:  -19.21 L
-------------------
Total Booked + Current PnL: 20.98 L (17.24%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.76%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.92 L (58.32%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.66%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.75,57.0,H-LC,11.05,172392.0,16215.0,9878.0,0.25,10.38,5.73,16.71,22.0,1.64,1.23,29.63,X40N,BTT,HEALTHCARE
63,SBIN,863.00,-11.34,62.0,M-LC,4.22,218691.0,17212.0,9994.0,0.06,8.54,4.57,13.51,60.0,1.72,1.56,22.22,XY25,NTT,BANKS
78,TTKPRESTIG,770.00,104.70,59.0,M-SC,4.42,88228.0,-12549.0,12643.0,0.20,-12.45,14.33,0.09,245.0,-0.99,0.63,15.91,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.83,68.0,H-MC,5.66,114014.0,13510.0,17125.0,-0.77,13.44,15.02,30.49,79.0,0.79,0.81,50.20,MH,ATH,METALS
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202250.0,2112.0,18769.0,0.23,1.06,9.28,10.44,4.0,0.11,1.44,6.68,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-29.69,55.0,H-SC,17.31,200999.0,1045.0,96178.0,0.62,0.52,47.85,48.63,115.0,0.01,1.44,12.14,XR,NTT,MISC
35,ICICIPRULI,790.00,-20.88,44.0,H-MC,2.23,136188.0,400.0,42354.0,-0.43,0.29,31.10,31.48,107.0,0.01,0.97,12.45,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.34,62.0,H-MC,1.79,248851.0,1120.0,73934.0,1.44,0.45,29.71,30.30,92.0,0.02,1.78,13.80,X40,ATH,ELECTRICAL
12,BANKINDIA,190.00,-30.43,61.0,H-MC,5.74,180623.0,815.0,111589.0,0.09,0.45,61.78,62.52,88.0,0.01,1.29,34.69,XR,NTT,BANKS
52,PGHH,17973.08,-31.08,51.0,H-MC,4.38,199560.0,-1260.0,70046.0,0.42,-0.63,35.10,34.25,80.0,-0.02,1.43,4.54,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-16.43,38.0,H-LC,5.81,300913.0,-14929.0,127587.0,0.39,-4.73,42.40,35.67,5.0,-0.12,2.15,8.47,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-12.80,48.0,H-LC,8.41,220229.0,-31539.0,84744.0,-1.72,-12.53,38.48,21.13,27.0,-0.37,1.57,18.46,X40,ATH,PAINTS
31,HCLTECH,1943.91,1.08,49.0,H-LC,10.02,230884.0,-11040.0,74298.0,0.28,-4.56,32.18,26.15,8.0,-0.15,1.65,9.11,X40,ATH,IT
44,JIOFIN,387.00,-0.18,50.0,H-LC,13.45,271201.0,3836.0,61617.0,1.33,1.43,22.72,24.48,48.0,0.06,1.94,57.20,XY24,BTT,FINANCE
20,CIPLA,1795.00,-20.12,50.0,H-LC,5.80,213062.0,8562.0,32854.0,0.45,4.19,15.42,20.25,10.0,0.26,1.52,13.29,X40N,BTT,PHARMA


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-2.60,63.0,H-LC,1.01,241830.0,-4321.0,83528.0,0.61,-1.76,34.54,32.18,16.0,-0.05,1.73,33.65,X200,ATH,IT
42,ITC,452.0,-37.50,54.0,H-LC,2.86,202250.0,2112.0,18769.0,0.23,1.06,9.28,10.44,4.0,0.11,1.44,6.68,X40,NTT,FMCG
51,NESTLEIND,1377.0,-8.22,58.0,H-LC,4.32,282274.0,16848.0,39942.0,-0.51,6.35,14.15,21.40,11.0,0.42,2.02,14.63,XY25,NTT,FMCG
57,RELIANCE,1533.0,-12.54,56.0,H-LC,5.59,219258.0,8892.0,19887.0,0.44,4.23,9.07,13.68,37.0,0.45,1.57,21.45,XY25,NTT,REFINERIES
20,CIPLA,1795.0,-20.12,50.0,H-LC,5.80,213062.0,8562.0,32854.0,0.45,4.19,15.42,20.25,10.0,0.26,1.52,13.29,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,66.94,72.0,H-SC,8.42,144717.0,-38058.0,149131.0,7.82,-20.82,103.05,60.77,125.0,-0.26,1.03,29.62,XR,NTT,JEWELLERY
22,COFFEEDAY,80.00,-39.64,57.0,L-SC,42.04,88493.0,-25056.0,65104.0,3.76,-22.07,73.57,35.27,268.0,-0.38,0.63,116.59,XR,NTT,HOTELS
8,ATULAUTO,844.00,3916.22,65.0,M-SC,6.44,125245.0,-22123.0,73945.0,2.48,-15.01,59.04,35.16,236.0,-0.30,0.89,27.93,XY24,NTT,AUTO
77,TRIDENT,48.00,-5.84,62.0,M-SC,5.38,73236.0,-19018.0,45326.0,2.24,-20.62,61.89,28.51,224.0,-0.42,0.52,25.32,XR,NTT,TEXTILES
55,RAJOOENG,143.33,-34.52,43.0,H-SC,10.66,99330.0,-3170.0,44003.0,2.19,-3.09,44.30,39.83,114.0,-0.07,0.71,3.83,AR,ATH,MISC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SHALBY,327.00,1064.27,65.0,M-SC,2.56,158434.0,-22709.0,68174.0,-1.77,-12.54,43.03,25.10,235.0,-0.33,1.13,26.97,XY24,NTT,HEALTHCARE
6,ASIANPAINT,3465.66,-12.80,48.0,H-LC,8.41,220229.0,-31539.0,84744.0,-1.72,-12.53,38.48,21.13,27.0,-0.37,1.57,18.46,X40,ATH,PAINTS
27,GREENPANEL,537.00,237.21,66.0,M-SC,4.74,150475.0,-32603.0,111577.0,-1.61,-17.81,74.15,43.14,230.0,-0.29,1.07,41.54,XY24,NTT,MISC
76,TITAGARH,1548.00,-0.35,65.0,H-SC,4.17,163328.0,-28687.0,107568.0,-1.58,-14.94,65.86,41.08,158.0,-0.27,1.17,38.40,XY24,NTT,ENGINEERING
4,ANGELONE,3033.00,3.06,35.0,H-SC,9.27,182516.0,-8490.0,66199.0,-1.16,-4.44,36.27,30.21,157.0,-0.13,1.30,15.51,X40N,NTT,FINANCE


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,98.37,59.0,H-SC,12.44,125307.0,-17172.0,36690.0,-0.66,-12.05,29.28,13.70,163.0,-0.47,0.90,47.90,OX40N,NTT,BANKS
78,TTKPRESTIG,770.0,104.70,59.0,M-SC,4.42,88228.0,-12549.0,12643.0,0.20,-12.45,14.33,0.09,245.0,-0.99,0.63,15.91,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.61,59.0,H-SC,1.74,223200.0,-46467.0,82807.0,0.75,-17.23,37.10,13.47,138.0,-0.56,1.59,12.89,XY24,NTT,PAINTS
19,CERA,9475.0,-20.29,44.0,H-SC,2.16,145337.0,-30566.0,72581.0,-0.05,-17.38,49.94,23.89,149.0,-0.42,1.04,25.52,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-20.18,52.0,H-MC,7.82,107948.0,-23887.0,66107.0,-0.06,-18.12,61.24,32.03,98.0,-0.36,0.77,20.91,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-26.44,53.0,H-LC,12.83,293750.0,-52206.0,127076.0,0.42,-15.09,43.26,21.64,1.0,-0.41,2.10,4.06,X40,ATH,IT
40,INFY,2275.00,-17.80,52.0,H-LC,8.82,320458.0,7570.0,164107.0,-0.26,2.42,51.21,54.87,3.0,0.05,2.29,9.23,X40,BTT,IT
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202250.0,2112.0,18769.0,0.23,1.06,9.28,10.44,4.0,0.11,1.44,6.68,X40,NTT,FMCG
84,VBL,671.64,-16.43,38.0,H-LC,5.81,300913.0,-14929.0,127587.0,0.39,-4.73,42.40,35.67,5.0,-0.12,2.15,8.47,X40N,ATH,FMCG
1,ABB,7934.00,-38.60,62.0,H-LC,11.13,256454.0,-5165.0,124380.0,0.07,-1.97,48.50,45.57,7.0,-0.04,1.83,9.12,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,64.0,M-MC,6.96,227872.0,2910.0,164751.0,-0.35,1.29,72.30,74.53,192.0,0.02,1.63,33.96,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,11.32,59.0,H-MC,3.69,192140.0,27470.0,40503.0,1.71,16.68,21.08,41.28,89.0,0.68,1.37,26.00,X40N,ATH,AC
85,VOLTAS,1918.49,2.42,62.0,H-MC,2.66,214830.0,23088.0,72935.0,1.16,12.04,33.95,50.08,99.0,0.32,1.53,19.66,XY25,ATH,AC
30,HAVELLS,2069.16,0.34,62.0,H-MC,1.79,248851.0,1120.0,73934.0,1.44,0.45,29.71,30.30,92.0,0.02,1.78,13.80,X40,ATH,ELECTRICAL
24,DABUR,735.00,-3.12,57.0,H-MC,6.57,210698.0,13670.0,75957.0,-0.12,6.94,36.05,45.49,102.0,0.18,1.51,20.88,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.83,68.0,H-MC,5.66,114014.0,13510.0,17125.0,-0.77,13.44,15.02,30.49,79.0,0.79,0.81,50.20,MH,ATH,METALS
86,WHIRLPOOL,2270.00,-39.28,59.0,M-SC,0.62,102712.0,11214.0,67533.0,1.93,12.26,65.75,86.07,223.0,0.17,0.73,49.16,XR,NTT,DURABLES
37,INDIAMART,4850.92,-50.72,52.0,H-SC,12.19,137577.0,14241.0,119527.0,-0.51,11.55,86.88,108.45,119.0,0.12,0.98,36.48,AR,ATH,MISC
87,WIPRO,420.00,-11.52,52.0,M-LC,5.61,155738.0,4793.0,104656.0,-0.04,3.18,67.20,72.51,53.0,0.05,1.11,9.39,XR,NTT,IT
36,IEX,219.00,-29.69,55.0,H-SC,17.31,200999.0,1045.0,96178.0,0.62,0.52,47.85,48.63,115.0,0.01,1.44,12.14,XR,NTT,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-5.01,46.0,H-SC,13.32,88477.0,-12486.0,107013.0,-0.18,-12.37,120.95,93.63,148.0,-0.12,0.63,31.78,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,57.0,L-SC,42.04,88493.0,-25056.0,65104.0,3.76,-22.07,73.57,35.27,268.0,-0.38,0.63,116.59,XR,NTT,HOTELS
49,MASFIN,398.61,-15.99,56.0,H-SC,12.91,95340.0,-2640.0,24245.0,-0.66,-2.69,25.43,22.05,152.0,-0.11,0.68,38.58,XR,ATH,FINANCE
50,NATIONALUM,247.44,-42.83,68.0,H-MC,5.66,114014.0,13510.0,17125.0,-0.77,13.44,15.02,30.49,79.0,0.79,0.81,50.20,MH,ATH,METALS
46,KPIGREEN,731.05,3.26,45.0,H-SC,10.25,123140.0,-2157.0,61090.0,1.69,-1.72,49.61,47.03,141.0,-0.04,0.88,54.18,MH,ATH,POWER


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-39.64,57.0,L-SC,42.04,88493.0,-25056.0,65104.0,3.76,-22.07,73.57,35.27,268.0,-0.38,0.63,116.59,XR,NTT,HOTELS
8,ATULAUTO,844.0,3916.22,65.0,M-SC,6.44,125245.0,-22123.0,73945.0,2.48,-15.01,59.04,35.16,236.0,-0.30,0.89,27.93,XY24,NTT,AUTO
82,VAIBHAVGBL,521.0,66.94,72.0,H-SC,8.42,144717.0,-38058.0,149131.0,7.82,-20.82,103.05,60.77,125.0,-0.26,1.03,29.62,XR,NTT,JEWELLERY
27,GREENPANEL,537.0,237.21,66.0,M-SC,4.74,150475.0,-32603.0,111577.0,-1.61,-17.81,74.15,43.14,230.0,-0.29,1.07,41.54,XY24,NTT,MISC
61,SAMMAANCAP,326.0,-170.45,61.0,M-SC,2.64,81372.0,-20838.0,114230.0,-0.14,-20.39,140.38,91.37,208.0,-0.18,0.58,32.09,XY25,NTT,FINANCE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.47
1,25,42.78
2,50,73.14


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.25
LC    33.94
MC    22.74
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.51
X40      14.99
X40N     12.75
XR       11.40
XY25      9.63
AR        8.50
OX40N     8.05
X200      1.73
MH        1.69
X5K       1.42
SR        1.26
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.39
H-LC    25.85
H-MC    19.83
M-SC    13.33
M-LC     7.04
M-MC     2.61
L-SC     1.53
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.13,-14.43,76.83
FMCG,11.64,-3.96,40.34
FINANCE,9.44,-16.28,61.83
BANKS,7.63,-12.33,64.04
MISC,7.38,-11.13,72.94
PAINTS,5.77,-14.19,31.21
ELECTRICAL,5.58,-5.36,63.43
AC,3.60,4.08,28.40
INSURANCE,3.35,-7.15,43.90


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2851131.0,22
XR,1311599.0,14
AR,1303839.0,9
X40,750007.0,10
X40N,626343.0,10
OX40N,520413.0,10
XY25,403610.0,7
SR,245192.0,2
X5K,118185.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3436090.0,29
M-SC,1373812.0,17
H-LC,1256044.0,15
H-MC,1177123.0,15
M-LC,391700.0,5
M-MC,313318.0,2
L-SC,245798.0,3
L-MC,60321.0,1
L-LC,37856.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1039092.0      6
M-SC       XY24       794178.0      7
H-SC       AR         784951.0      5
           XR         759860.0      7
H-MC       XY24       484061.0      4
H-LC       X40        468994.0      5
           AR         357136.0      2
M-MC       XY24       313318.0      2
H-SC       X40N       297510.0      4
           OX40N      248395.0      4
           SR         245192.0      2
H-LC       X40N       224940.0      4
H-MC       X40        212647.0      4
           XY25       181701.0      2
L-SC       XR         162314.0      2
M-SC       AR         161752.0      2
M-LC       XY24       158865.0      2
M-SC       OX40N      122427.0      4
M-LC       X5K        118185.0      1
M-SC       XY25       114230.0      1
           XR         112859.0      2
H-MC       XR         111589.0      1
M-LC       XR         104656.0      1
H-MC       X40N       103893.0      2
H-LC       X200        83528.0      1
L-SC       OX40N       83484.0      1
M-SC       X40         68366.0      1
H-MC       OX40N       66107.0      1
H-LC       XY24        61617.0      1
H-SC       MH          61090.0      1
L-MC       XR          60321.0      1
H-LC       XY25        59829.0      2
L-LC       XY25        37856.0      1
H-MC       MH          17125.0      1
M-LC       XY25         9994.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
